# Learning the spatial structure of Fire Weather Index with convolutional neural networks

***Environmental Research Letters***

**O. Mirones, J. Baño-Medina, J. Bedia**

This notebook reproduces the results presented in the manuscript entitled **Learning the spatial structure of Fire Weather Index with convolutional neural networks**, submitted to the journal *Environmental Research Letters* by *O. Mirones, J. Baño-Medina, J. Bedia*. 
This paper analyzes the ability of state-of-the-art machine learning algorithms, especially convolutional neural networks, to model the spatial structure of the Fire Weather Index (FWI).

## Table of contents:
*  [1 Preparing the R environment](#1-bullet)
*  [2  Load data](#2-bullet)
    *  [2.1  Load predictor data](#2.1-bullet)
    *  [2.2  Load predictand data](#2.2-bullet)
*  [3  Downscaling: Cross-validation](#3-bullet)
    *  [3.1  Analogs](#3.1-bullet)
    *  [3.2  Generalized Linear Models](#3.2-bullet)
    *  [3.3  Convolutional neural network multi-site](#3.3-bullet)
    *  [3.4  Convolutional neural network multi-site gaussian](#3.4-bullet)
    *  [3.5  Convolutional neural network multi-site multi-gaussian](#3.5-bullet)
*  [4  Results](#4-bullet)
    *  [4.1  Correlograms](#4.1-bullet)
    *  [4.2  Mutual Information](#4.2-bullet)

<div class="alert alert-block alert-info">
<b>Note:</b> This notebook was run on a machine with the following technical specifications: !!!ESTÁ HECHO EN LUSTRE: PREGUNTAR A ANTONIO Y ZEQUI!!!

- Operating system: Ubuntu 18.04.3 LTS (64 bits)
- Memory: 60 GiB
- Processor: 2x Intel(R) Xeon(R) CPU E5-2670 0 @ 2.60GHz (16 cores, 32 threads)
</div>

## 1. Preparing the R environment <a class="anchor" id="1-bullet"></a>

In particular, the following C4R libraries are used along the notebook:


 * `loadeR` (v1.7.0) for data loading,
 * `transformeR` (v2.1.0) for data manipulation, 
 * [`downscaleR`](https://doi.org/10.5194/gmd-13-1711-2020) (v3.3.2) for downscaling and
 * [`downscaleR.keras`](https://doi.org/10.5194/gmd-13-2109-2020) (v1.0.0) for downscaling with neural networks and

A frozen version of the above libraries and all the ones needed to reproduce the manuscript are installable through the `environment.yml` file using the `mamba` package, by executing the following commands in your command shell terminal: 
```shell
$ mamba env create -n deep-fwi --file=environment.yml
```

Once installed, you activate the newly created `deep-fwi` environment using `conda` by typing the following command:
```shell
$ conda activate deep-fwi
```

Once in the environment, type `R` and load the libraries:

In [ ]:
### Loading libraries
library(loadeR)
library(transformeR)
library(downscaleR)
library(magrittr)
library(downscaleR.keras)
library(tfprobability)
library(MASS)
library(VALUE)
library(geosphere)
library(scales)
library(abind)
library(gridExtra)
library(lattice)

We load some auxiliary functions from the `utils` directory. The file `./utils/topologiesCNN.R` contains the code implementing the convolutional neural networks used in this study, which are based on tensorflow-keras. The file `./utils/downscaleCV.keras.tfprobability.R` is a function enabling cross-validation over a convolutional neural network coded with the library tensorflow-probability. The file `./utils/dimFix.R` completes missing dimensions of VALUE objects. The file `./utils/corrMat.VALUE.R`computes the cross correlation matrices between stations that serve as input for plotting functions. These two files are a variation of its namesake from the package `VALUE`, adapted for use with notebook data structures. The file `./utils/corrL_allST.R` returns the correlation length for the `corrMat.VALUE` output. The `corrL_allST` function uses the file `./utils/corr_length.R` to calculate the correlation length based on the smoothed fit to the given data. Here, the `./utils/smooth_corr.R` smooths using a loess filter with standard settings. Finally, the file `./utils/miMat.VALUE.R`computes the mutual information between stations. These files is a variation of ist namesake from the package `VALUE`, adapted for the Fire Weather Index (`FWI`).

In [ ]:
source("./utils/topologiesCNN.R")
source("./utils/downscaleCV.keras.tfprobability.R")
source("./utils/dimFix.R")
source("./utils/corrMat.VALUE.R")
source("./utils/corrL_allST.R")
source("./utils/corr_length.R")
source("./utils/smooth_corr.R")
source("./utils/miMat.VALUE.R")

Below we define parameters related to the cross-validation procedure. 

In [ ]:
folds <- list(1985:1991, 1992:1998, 1999:2004, 2005:2011)
vars <- c("hur850", "ta850", "tas", "ua850", "va850")

## 2 Load data <a class="anchor" id="2-bullet"></a>
In this section we illustrate how to load the data into our `R` environment. Previously, we need to download the data from the Zenodo repository (DOI:) and store it in our working directory. For the notebook to work succesfully, it is important to preserve the structure of directories encountered in Zenodo. In addition we create a new folder where we will store the estimated fields named `./data/predictions/`. The folder `figures` is created to store the figures of the results obtained. 

In [ ]:
### Store in "./data/" the predictor and predictand files from Zenodo (DOI: ) such that you end up having "./data/predictors/", "./data/predictands/" and "./figures/" 
if (!dir.exists("./data/")) dir.create("./data/", showWarnings = FALSE)
###
if (!dir.exists("./data/predictions/")) dir.create("./data/predictions/", showWarnings = FALSE)
###
if (!dir.exists("./figures/")) dir.create("./figures/", showWarnings = FALSE)

## 2.1 Load predictor data <a class="anchor" id="2.1-bullet"></a>
We load the following predictor variables: relative humidity at 850hPa (`hur850`), air temperature at 850 hPa (`ta850`), air surface temperature (`tas`), zonal wind velocity at 850 hPa (`ua850`), and meridional wind velocity at 850 hPa (`va850`). These variables are stored as `.nc` files in the directory `./data/predictors/`. We use `loadGridData` from library `loadeR` to load the data into our `R` session. Then we call function `makeMultiGrid` to bind these predictor variables into a single object named `x`.

In [ ]:
### Loading predictor data
x <- lapply(vars, FUN = function(var) {
  out <- loadGridData(paste0("./data/predictors/", var, ".nc"), var = var)
  print(sprintf("Variable: %s loaded!", var))
  return(out)
}) %>% makeMultiGrid()

## 2.2 Load predictand data <a class="anchor" id="2.1-bullet"></a>

In [ ]:
### Loading predictand data (previously generated in 1_loadObservations.R)
y <- load("./Rdata/AEMET_SUBSET/AEMET_13UTC_fwi13.Rdata") %>% get() %>% subsetGrid(years = 1985:2011)

## 3 Downscaling: Cross-validation <a class="anchor" id="3-bullet"></a>
In this section we perform cross-validation over the machine learning methods used in this study: analogs, generalized linear models, and 3 different convolutional neural network architectures. Therefore, we split the data into the following 4 chronological folds: 1985-1991, 1992-1998, 1999-2004, 2005-2011. We store the resulting predictions into the previously created `./data/predictions/` folder. These files will be used in the next section. 

In [ ]:
pred.dir <- "./data/predictions/"

The results presented in this section are predominantly based on libraries `downscaleR`, `transformeR` and `downscaleR.keras`. In particular we use the function `downscaleCV` from library `downscaleR` to perform cross-validation for the analogs and generalized linear models. Similary, we use the function `downscaleCV.keras` from `downscaleR.keras` to cross-validate the convolutional neural networks. The only exception is the convolutional model multi-site multi-gaussian which builds on function `downscaleCV.keras.tfprobability` located in the `utils` directory.

## 3.1 Analogs <a class="anchor" id="3.1-bullet"></a>

In [ ]:
### Downscaling analogs
p <- downscaleCV(x = x,
                 y = y,
                 method = "analogs",
                 n.analogs = number_analogs,
                 sel.fun = "mean",
                 window = 7,
                 scaleGrid.args = list(type = "standardize"),
                 folds = folds)
predFileName <- sprintf("%spred_analogs_AEMET_13UTC_fwi13.Rdata", pred.dir)
save(p, file = predFileName)

## 3.2 Generalized Linear Model <a class="anchor" id="3.2-bullet"></a>

In [ ]:
### Downscaling generalized linear model
number_of_neighbours = 16
p <- downscaleCV(x = x,
                 y = y,
                 method = "GLM",
                 family = "gaussian",
                 prepareData.args = list("local.predictors" = list(n = number_of_neighbours, vars = getVarNames(x))),
                 scaleGrid.args = list(type = "standardize"),
                 folds = folds)
predFileName <- sprintf("%spred_glm%s_AEMET_13UTC_fwi13.Rdata", pred.dir, number_of_neighbours)
save(p, file = predFileName)

## 3.3 Convolutional neural network multi-site <a class="anchor" id="3.3-bullet"></a>

In [ ]:
### Downscaling convolutional neural network multi-site
model <- cnn_model(topology = "cnn-multi-site",
                   input_shape = c(getShape(x, "lat"), getShape(x, "lon"), getShape(x, "var")),
                   output_shape = getShape(y, "loc"),
                   kernel_size = c(3,3),
                   neurons = c(50, 50))
p <- downscaleCV.keras(x = x,
                       y = y,
                       model = model,
                       prepareData.keras.args = list(first.connection = "conv", last.connection = "dense", channels = "last"),
                       compile.args = list("loss" = "mse", "optimizer" = optimizer_adam(lr = 0.0001)),
                       fit.args = list("batch_size" = 100, "epochs" = 10000, "validation_split" = 0.1, "verbose" = 0, "callbacks" = list(callback_early_stopping(patience = 30))),
                       scaleGrid.args = list(type = "standardize"),
                       folds = folds)
predFileName <- sprintf("%spred_cnn-multi-site_AEMET_13UTC_fwi13.Rdata", pred.dir)
save(p, file = predFileName)

## 3.4 Convolutional neural network multi-site gaussian <a class="anchor" id="3.4-bullet"></a>

In [ ]:
### Downscaling convolutional neural network multi-site gaussian
model <- cnn_model(topology = "cnn-multi-site-gaussian",
                   input_shape = c(getShape(x, "lat"), getShape(x, "lon"), getShape(x, "var")),
                   output_shape = getShape(y, "loc"),
                   kernel_size = c(3,3),
                   neurons = c(50, 50))
p <- downscaleCV.keras(x = x,
                       y = y,
                       model = model,
                       loss = "gaussianLoss",
                       prepareData.keras.args = list(first.connection = "conv", last.connection = "dense", channels = "last"),
                       compile.args = list("loss" = gaussianLoss(last.connection = "dense"), "optimizer" = optimizer_adam(lr = 0.0001)),
                       fit.args = list("batch_size" = 100, "epochs" = 10000, "validation_split" = 0.1, "verbose" = 0, "callbacks" = list(callback_early_stopping(patience = 30))),
                       scaleGrid.args = list(type = "standardize"),
                       folds = folds)
p <- lapply(1:30, FUN = function(member) {
  computeTemperature(mean = subsetGrid(p, var = "mean"), log_var = subsetGrid(p, var = "log_var"))
}) %>% bindGrid(dimension = "member")
attr(p$Data, "dimensions") <- c("member","time", "loc")
p <- binaryGrid(p, condition = "GT", threshold = 0, partial = TRUE)
predFileName <- sprintf("./Rdata/%s/pred_cnn-multi-site-gaussian_30samples_AEMET_13UTC_fwi13.Rdata", dataset)
save(p, file = predFileName)

## 3.5 Convolutional neural network multi-site multi-gaussian <a class="anchor" id="3.5-bullet"></a>

In [ ]:
### Downscaling convolutional neural network multi-site multi-gaussian
negloglik <- function (x, rv_x) - (rv_x %>% tfd_log_prob(x))
p <- downscaleCV.keras.tfprobability(x = x,
                                     y = y,
                                     samples = 30,
                                     cnn_model.args = list(topology = "cnn-multi-site-multi-gaussian",
                                                           input_shape = c(getShape(x, "lat"), getShape(x, "lon"), getShape(x, "var")),
                                                           output_shape = getShape(y, "loc"),
                                                           kernel_size = c(3,3),
                                                           neurons = c(200, 200)),
                                     type_nn = "multivariate-gaussian",
                                     prepareData.keras.args = list(first.connection = "conv", last.connection = "dense", channels = "last"),
                                     compile.args = list("loss" = negloglik, "optimizer" = optimizer_adam(lr = 0.0001)),
                                     fit.args = list("batch_size" = 100L, "epochs" = 10000L, "validation_split" = 0.1, "verbose" = 0L, "callbacks" = list(callback_early_stopping(patience = 30))),
                                     scaleGrid.args = list(type = "standardize"),
                                     folds = folds)
predFileName <- sprintf("./Rdata/%s/pred_cnn-multi-site-multi-gaussian_30samples_AEMET_13UTC_fwi13.Rdata", dataset)
save(p, file = predFileName)

## 4 Results <a class="anchor" id="4-bullet"></a>

In this section, we will present an analysis of the results obtained from the different predictions. Firstly, we will display the correlograms that have been constructed for each prediction. Following that, we will present the mutual information matrices.

## 4.1 Correlograms <a class="anchor" id="4.1-bullet"></a>

Here, we assess the strength of linear relationships between observed and predicted `FWI` time series among different locations. We accomplish this by calculating pairwise *Spearman's* cross-correlations among all pairs of stations. To visualize these relationships, we construct a location plot by plotting the cross-correlation value for each station pair against their respective geographical distances.

Next, we fit a local 2nd-order polynomial (loess) to obtain two curves—one for predictions and one for observations. This enables a straightforward visual assessment of the spatial correlation structure of predictions compared to the reference observations.

For the analysis, we define the fire season as June to September (JJAS), and we set a threshold ($0.4$) for the correlation length and specify the correlation type. The correlations are then computed using the `corrL_allST` function.

Additionally, we calculate the Root Mean Square Error (`RMSE`), Mean Absolute Error (`MAE`), and the correlation length bias (`CL bias`). These metrics provide valuable insights into the performance of the predictions compared to the reference observations. All the computed values, including correlations, correlation length, `RMSE`, `MAE`, and `CL bias`, are included in the correlation plots.

To facilitate easy access and reference, the resulting figure will be stored in the `figures` directory.

In [ ]:
###Defining season, threshold and correlation type

sea <- "JJAS"
tr <- .4
corrtype <- "spearman"

In [ ]:
###Computing and plotting the correlogram for the reference observations
pdf("./figures/correlograms.pdf", width = 14, height = 8.5)
par(mfrow = c(2,3))

corrL_obs <- corrL_allST(data=y, type = "after", corrtype = corrtype,tr=tr,season=sea)

plot(corrL_obs$JJAS, col = alpha('grey', 0.3), pch = 16, ylim = c(-0.3,1),
     xlab = "Distance [km]", ylab = "Correlation", main = "Obs", cex.lab = 1.5, cex.main = 1.5, cex.axis = 1.5,
     las = 1)
abline(h = tr, lty = 2)
abline(v = corrL_obs$JJA$corrL, lwd = 2, col = 'grey')
lines(corrL_obs$JJAS$x_fit, corrL_obs$JJAS$y_fit, col = 'grey', lwd = 2.5)
grid()

In [ ]:
###Computing and plotting the correlograms for the predictions
preds <- list.files(pred.dir)

for (i in c(1:length(preds))) {
  
  ###Loading prediction and computing the correlation length
  
  p <- load(paste0(pred.dir, preds[i]))%>% get()
  corrL_pred <- corrL_allST(data=p, type = "after", corrtype = corrtype,tr=tr,season=sea)
  
  ###Computing RMSE and MAE
  
  d <- corrL_obs$JJAS$y - corrL_pred$JJAS$y
  mae <- mean(abs(d))
  mse <- mean((d)^2)
  rmse <- sqrt(mse)
  bias.corrL <- corrL_obs$JJAS$corrL-corrL_pred$JJAS$corrL
  
  ###Visualization
  plot(corrL_pred$JJAS, col = alpha('red', 0.3), pch = 16, ylim = c(-0.3,1),
       xlab = "Distance [km]", ylab = "Correlation",
       main = gsub("^pred_(.*?)_AEMET.*$", "\\U\\1", preds[i], perl = TRUE), 
       cex.lab = 1.5, cex.main = 1.5, cex.axis = 1.5, las = 1)
  abline(h = tr, lty = 2)
  abline(v = corrL_obs$JJAS$corrL, lwd = 2, col = 'grey')
  lines(corrL_obs$JJAS$x_fit, corrL_obs$JJAS$y_fit, col = 'grey', lwd = 2.5) 
  abline(v = corrL_pred$JJAS$corrL, lwd = 2, col = 'red')
  lines(corrL_pred$JJAS$x_fit, corrL_pred$JJAS$y_fit, col = 'red', lwd = 2.5)
  text(1225,0.85, paste0("RMSE: ", round(rmse,3)), cex = 1.45)
  text(1225,0.75, paste0("MAE: ", round(mae,3)), cex = 1.45)
  text(1225,0.95, paste0("CL bias: ", round(bias.corrL, 3)), cex = 1.45)
  grid()
 
}

dev.off()

## 4.2 Mutual Information <a class="anchor" id="4.2-bullet"></a>

We proceed to plot each mutual information (`MI`) value, denoted as $M_{i,j}$, against the distance between the corresponding locations, i.e., $i$ and $j$. To capture the underlying patterns, we fit a second-degree loess curve to these plots.

To determine the mutual information lengths (`MIL`), we establish MI thresholds for both the observations and the different downscaling methods. In this analysis, we utilize a MI threshold of $0.05$. This threshold allows for comparable results to correlation length analyses and aids in the identification of potential new information regarding the performance of each method.

Based on the climatic regions, the stations are grouped together in a matrix. This matrix exhibits three primary groupings, each with a quadrilateral shape. These groupings correspond to the following climatic regions: the *Atlantic* region, the *Continental Mediterranean* region, and the *Coastal Mediterranean* stations.

In [ ]:
###Defining the order of the stations

st.names <- c("REUS","S.COMPOSTELA","VIGO","SORIA","VALLADOLID","ZAMORA","LEON","SALAMANCA",
               "MADRID-BARAJAS","MADRID-RETIRO","CIUDAD REAL","BADAJOZ","GRANADA",
               "SEVILLA","MORON","JEREZ","ALMERIA","MURCIA","ALICANTE-ALTET",
               "ALICANTE","CUENCA","VALENCIA-AER.","VALENCIA","LOGRONO","DAROCA","TORTOSA",
               "MALLORCA","MENORCA","IBIZA")

y$Metadata$name <- st.names

###Defining the FWI90 event for the mutual information

prob <- .9

In [ ]:
###defining the color palettes

mi.colors <- RColorBrewer::brewer.pal(11, "Reds") %>% colorRampPalette()
spec.colors <- RColorBrewer::brewer.pal(11, "Spectral") %>% colorRampPalette()

In [ ]:
#modification of stations order
ind_station.ids <- c(2:10,21,24,13,12,11,25,1,27:29,18:20,22,23,26,14:17)
station.ids <- y$Metadata$station_id[ind_station.ids]
y <- subsetGrid(y, station.id = station.ids) %>% redim(member = FALSE, loc = TRUE)

In this step, we compute the `MI` matrix for both the reference observations and the predictions. To perform these computations, we utilize the `miMat.VALUE` function. This function enables us to calculate the `MI` values and construct the `MI` matrix, which represents the pairwise `MI` between different locations.

In [ ]:
### Mutual Information matrix computation for the observations

mi.matrix.obs <- miMat.VALUE(y,
                         predictionObj = NULL,
                         season = "JJAS",
                         threshold = NULL,
                         prob = prob)


idx <- grep("analogs", preds, invert = T)
preds <- preds[idx]

In [ ]:
### Mutual Information matrix computation for the predictions

mi.matrix.list <- list()
for (i in c(1:length(preds))) {

  ### Loading predictions
  
  p <- load(paste0(pred.dir, preds[i]))%>% get()
  
  ###Ordering stations by location
  
  p <- subsetGrid(p, station.id = station.ids) %>% redim(member = FALSE, loc = TRUE)
  
  ###Computing the Mutual Information matrix
  
  mi.matrix <- miMat.VALUE(p,
                predictionObj = NULL,
                season = "JJAS",
                threshold = NULL,
                prob = prob)
  
  mi.matrix.list[[i]] <- mi.matrix
  
  
}

Lastly, we proceed to plot the `MI` matrices for the `FWI90` event during the fire season (June to September). The first matrix computed represents the MI for the reference observations, displayed in the upper triangle, and the best-performing model (`CNN-MSMG`) in the lower triangle. The bottom left and right matrices depict the `MI` bias of the methods compared to the observations.

In the `MI` bias matrices, the analogs method is excluded from the representation since `MI` biases are negligible by construction. These matrices are divided to show the bias corresponding to the different methods in the upper and lower triangles. Only pairs of stations with `MI` values $\ge 0.05$ for the observations are displayed in the `MI` bias matrices.

In [ ]:
scales.list <- list(x = list(labels = attributes(mi.matrix.obs[[1]])$station_names, rot = 90,
                             at = seq(1,ncol(mi.matrix.obs[[1]]),1), cex = .65),
                    y = list(labels = attributes(mi.matrix.obs[[1]])$station_names,
                             at = seq(1,ncol(mi.matrix.obs[[1]]),1), cex = .65),
                   alternating = 2)

In [ ]:
###Computing the first matrix: the upper diagonal contains the mutual information for the obs;
###the lower diagonal is for the cnn-multi-site-multi-gaussian

new.mi.matrix <- mi.matrix.obs

idx.lower.tri <- which(lower.tri(new.mi.matrix$JJAS), arr.ind = T)

new.mi.matrix$JJAS[idx.lower.tri] <- mi.matrix.list[[grep("cnn-multi-site-multi-gaussian", preds)]]$JJAS[idx.lower.tri] #changing the lower triangle

l1 <- levelplot(new.mi.matrix$JJAS, ylab = "AEMET_13UTC_FWI13 (Obs)" ,
                xlab = "CNN-MULTI-SITE-MULTI-GAUSSIAN",
                scales = scales.list, main = "", col.regions = mi.colors(100),
               colorkey = list(space = "bottom", height = 1, width = 1)) 

In [ ]:
###Replacing with NAs those values with MI < 0.05 (below the threshold defined)

idx.na <- which(mi.matrix.obs$JJAS < 0.05, arr.ind = T)


names <- gsub("^pred_(.*?)_AEMET.*$", "\\U\\1", preds, perl = TRUE)

levelplot.list <- list()
for (i in c(1,3)) {
  
  new.mi.matrix <- mi.matrix.list[[i]]
  idx.lower.tri <- which(lower.tri(new.mi.matrix$JJAS), arr.ind = T)
  new.mi.matrix$JJAS[idx.lower.tri] <- mi.matrix.list[[i+1]]$JJAS[idx.lower.tri]
  
  new.mi.matrix$JJAS[idx.na] <- NA
  
  l <- levelplot(new.mi.matrix$JJAS-mi.matrix.obs$JJAS, ylab = names[i], xlab = names[i+1], scales = scales.list,
                 main = "", col.regions = rev(spec.colors(100)), at = seq(-0.15,0.15,0.01),
                 colorkey = list(space = "bottom", height = 1, width = 1)) 
  
  levelplot.list[[length(levelplot.list)+1]] <- l
}

pdf("./figures/MI_Matrices.pdf", width = 22.5, height = 11)

grid.arrange(l1, levelplot.list[[1]], levelplot.list[[2]], nrow = 2, ncol = 2)

dev.off()